In [ ]:
import pandas as pd


merged_df = pd.read_csv('/kaggle/input/merged-paired-dataset/dataset_generated_to_not.csv', index_col='Unnamed: 0').reset_index(drop=True)

In [ ]:
merged_df

In [ ]:
merged_df_short.columns

In [ ]:
merged_df_short = merged_df.loc[5501:6000]
merged_df_short

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, DebertaTokenizer, DebertaConfig
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DebertaTokenizer, DebertaForSequenceClassification, DebertaConfig

config_deberta = DebertaConfig.from_pretrained("deepvk/deberta-v1-distill")
model_state_deberta = torch.load("/kaggle/input/deberta-pre-trained/pytorch/v1/1/model_deberta_v2.pth")
tokenizer = DebertaTokenizer.from_pretrained("deepvk/deberta-v1-distill")
model = DebertaForSequenceClassification(config_deberta)

model.load_state_dict(model_state_deberta)
model.eval()

def compute_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model.deberta(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

generated_clean_texts = merged_df_short['generated_text_clean'].tolist()
clean_texts = merged_df_short['clean_text'].tolist()

generated_embeddings = compute_embeddings(generated_clean_texts, tokenizer, model)
clean_embeddings = compute_embeddings(clean_texts, tokenizer, model)

similarities = [cosine_similarity([gen_emb], [clean_emb])[0][0] for gen_emb, clean_emb in zip(generated_embeddings, clean_embeddings)]

merged_df_short['cosine_similarity_calculation_'] = similarities

merged_df_short.to_csv('merged_df_with_similarity.csv', index=False)

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer,DistilBertConfig, DistilBertForSequenceClassification, Trainer, TrainingArguments

tmp = pd.DataFrame(columns=['title', 'generated_text', 'generated', 'model',
       'generated_text_without_title', 'generated_text_clean', 'clean_text',
       'text'])
tmp

In [ ]:


config_rubert = DistilBertConfig.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational")
model_state_rubert = torch.load("/kaggle/input/rubert-pre-trained/pytorch/v1/1/model_distilrubert_latest.pth")

model = DistilBertForSequenceClassification(config_rubert)

model.load_state_dict(model_state_rubert)
tokenizer = DistilBertTokenizer.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational")
model.eval()

def compute_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.distilbert(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

generated_clean_texts = merged_df_short['generated_text_clean'].tolist()
clean_texts = merged_df_short['clean_text'].tolist()

encodings_rubert_generated = compute_embeddings(generated_clean_texts, tokenizer, model)
encodings_rubert_writed = compute_embeddings(clean_texts, tokenizer, model)

similarities = [cosine_similarity([clean_emb], [gen_emb])[0][0] for gen_emb, clean_emb in zip(encodings_rubert_generated, encodings_rubert_writed)]

merged_df_short.loc[:, 'cosine_similarity_calculation_'] = similarities



In [ ]:
tmp = pd.concat([tmp, merged_df_short])
tmp.to_csv('merged_df_with_similarity.csv', index=False)

In [ ]:
tmp[tmp.cosine_similarity_calculation_ > 0].count()

In [ ]:
merged_df_short.loc[2524].generated_text_without_title

In [ ]:
merged_df_short.loc[2524].text

In [ ]:
merged_df_short[merged_df_short['cosine_similarity_calculation_'] > -0.1]